# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = pd.read_csv("../Output/weather_check.csv")
city_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Punta Arenas,-53.15,-70.92,46.40,70,75,9.17,CL,1602852059
1,1,Yulara,-25.24,130.99,86.00,17,0,8.05,AU,1602852060
2,2,Douglas,41.30,-96.17,36.00,86,40,5.82,US,1602852061
3,3,Xiaoweizhai,26.19,107.51,52.81,90,100,6.96,CN,1602851766
4,4,Vaini,-21.20,-175.20,75.20,94,90,5.82,TO,1602852064


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_weather[["Lat", "Lng"]]
humidity = city_weather["Humidity"]
max_humidity = humidity.max()

heat_map_figure = gmaps.figure(layout = {
        'width': '1200px',
        'height': '800px'})
heat_map_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = max_humidity, 
                                    point_radius = 3)

heat_map_figure.add_layer(heat_map_layer)

heat_map_figure

Figure(layout=FigureLayout(height='800px', width='1200px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
narrow_down_cities_df = city_weather.loc[(city_weather["Max Temp"].astype(float) >= 70.0) & 
                                         (city_weather["Max Temp"].astype(float) <= 80.0) & 
                                         (city_weather["Cloudiness"].astype(float) == 0.0) & 
                                         (city_weather["Wind Speed"].astype(float) < 10.0)]

narrow_down_cities_df = narrow_down_cities_df.dropna(how = "any")

del narrow_down_cities_df["Unnamed: 0"]

narrow_down_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,Sarakhs,36.54,61.16,77.00,23,0,6.93,IR,1602852082
74,Guerrero Negro,27.98,-114.06,71.13,71,0,9.42,MX,1602852148
89,Cabo San Lucas,22.89,-109.91,77.00,52,0,1.01,MX,1602852168
107,Çumra,37.57,32.77,77.00,23,0,2.24,TR,1602852191
186,Tezu,27.92,96.17,78.73,62,0,3.38,IN,1602852281
209,Río Tercero,-32.17,-64.11,73.99,43,0,1.01,AR,1602852308
325,Mercedes,-34.65,-59.43,70.00,74,0,3.00,AR,1602852457
334,Barquisimeto,10.07,-69.32,71.60,100,0,3.36,VE,1602852253
361,Mamit,23.93,92.48,75.42,90,0,1.16,IN,1602852502
396,Along,28.17,94.77,73.83,83,0,0.47,IN,1602852548


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(narrow_down_cities_df, columns=["City", "Country", "Lat", "Lng"])
hotels = []

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    params = {
        "location" : f"{latitude},{longitude}", 
        "radius" : 5000, 
        "types" : "hotel", 
        "key" : g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response =  requests.get(base_url, params = params)
    
    response_json = response.json()
    
    try:
        hotels.append(response_json['results'][0]['name'])
    except IndexError:
        hotels.append(np.nan)
    
    time.sleep(1)

In [8]:
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Country,Lat,Lng,Hotel Name
19,Sarakhs,IR,36.54,61.16,Sarakhs
74,Guerrero Negro,MX,27.98,-114.06,Guerrero Negro
89,Cabo San Lucas,MX,22.89,-109.91,Cabo San Lucas
107,Çumra,TR,37.57,32.77,Çumra
186,Tezu,IN,27.92,96.17,Tezu
209,Río Tercero,AR,-32.17,-64.11,Río Tercero
325,Mercedes,AR,-34.65,-59.43,Mercedes
334,Barquisimeto,VE,10.07,-69.32,Barquisimeto
361,Mamit,IN,23.93,92.48,Mamit
396,Along,IN,28.17,94.77,Nyingchi


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

heat_map_figure.add_layer(markers)

# Display figure
heat_map_figure

Figure(layout=FigureLayout(height='800px', width='1200px'))